In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hermanus,ZA,2021-01-31 21:02:49,-34.4187,19.2345,64.00,88,100,1.99
1,1,Saint George,US,2021-01-31 21:03:01,37.1041,-113.5841,55.99,53,1,3.80
2,2,Carballo,ES,2021-01-31 21:06:05,43.2130,-8.6910,55.99,80,96,13.00
3,3,Guaratinga,BR,2021-01-31 21:06:05,-16.5667,-39.5667,78.80,73,40,9.22
4,4,Cape Town,ZA,2021-01-31 21:05:07,-33.9258,18.4232,69.01,83,75,10.36


In [4]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
print(len(temps))

542


In [26]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating = True, max_intensity = 300, point_radius = 15)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [29]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Guaratinga,BR,2021-01-31 21:06:05,-16.5667,-39.5667,78.80,73,40,9.22
6,6,Arraial Do Cabo,BR,2021-01-31 21:05:55,-22.9661,-42.0278,84.20,73,40,14.97
9,9,Georgetown,MY,2021-01-31 21:04:28,5.4112,100.3354,78.80,83,20,3.44
17,17,Saint-Philippe,RE,2021-01-31 21:06:07,-21.3585,55.7679,78.80,69,0,6.91
19,19,Bambous Virieux,MU,2021-01-31 21:06:07,-20.3428,57.7575,81.00,78,75,12.66
22,22,Cap Malheureux,MU,2021-01-31 21:06:07,-19.9842,57.6142,81.00,74,60,7.00
26,26,Alenquer,BR,2021-01-31 21:05:10,-1.9417,-54.7383,82.40,83,0,5.75
27,27,Port-Gentil,GA,2021-01-31 21:01:36,-0.7193,8.7815,80.60,89,40,6.91
32,32,Axim,GH,2021-01-31 21:03:19,4.8699,-2.2405,79.72,86,41,9.91
33,33,Eten,PE,2021-01-31 21:06:08,-6.9081,-79.8642,78.80,57,0,25.32


In [30]:
preferred_cities_df.count()

City_ID       160
City          160
Country       160
Date          160
Lat           160
Lng           160
Max Temp      160
Humidity      160
Cloudiness    160
Wind Speed    160
dtype: int64

In [31]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Guaratinga,BR,78.80,-16.5667,-39.5667,
6,Arraial Do Cabo,BR,84.20,-22.9661,-42.0278,
9,Georgetown,MY,78.80,5.4112,100.3354,
17,Saint-Philippe,RE,78.80,-21.3585,55.7679,
19,Bambous Virieux,MU,81.00,-20.3428,57.7575,
22,Cap Malheureux,MU,81.00,-19.9842,57.6142,
26,Alenquer,BR,82.40,-1.9417,-54.7383,
27,Port-Gentil,GA,80.60,-0.7193,8.7815,
32,Axim,GH,79.72,4.8699,-2.2405,
33,Eten,PE,78.80,-6.9081,-79.8642,


In [33]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [38]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Guaratinga,BR,78.80,-16.5667,-39.5667,Hotel Caraíva
6,Arraial Do Cabo,BR,84.20,-22.9661,-42.0278,Pousada Porto Praia
9,Georgetown,MY,78.80,5.4112,100.3354,Cititel Penang
17,Saint-Philippe,RE,78.80,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
19,Bambous Virieux,MU,81.00,-20.3428,57.7575,Casa Tia Villa
22,Cap Malheureux,MU,81.00,-19.9842,57.6142,Pereybere Hotel & Spa
26,Alenquer,BR,82.40,-1.9417,-54.7383,Josilene Castro
27,Port-Gentil,GA,80.60,-0.7193,8.7815,Fengshui Residence
32,Axim,GH,79.72,4.8699,-2.2405,AXIM BEACH RESORT AND SPA
33,Eten,PE,78.80,-6.9081,-79.8642,Hospedaje Sol Radiante


In [42]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [43]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [44]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [45]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))